In [1]:
import glob
import os
from os.path import join

import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
def cross_validate(mfcc_list, labels_list, random_state_list, test_function, num_eigenvectors=1):
    results = {}
    accuracy_list = []
    accuracy_list_female = []
    accuracy_list_male = []
    for random_state in random_state_list:
        X_train, X_test, y_train, y_test = train_test_split(mfcc_list, labels_list, test_size=0.33,
                                                            random_state=random_state, stratify=labels_list)

        features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'F'))
        eigen_female = get_k_main_eigenvectors(features, k=num_eigenvectors)

        features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'M'))
        eigen_male = get_k_main_eigenvectors(features, k=num_eigenvectors)

        features_test = get_features_filtered_by_label(X_test, y_test, 'F')
        is_correct_function = lambda x, y: x < y
        accuracy_female = test_function(eigen_female, eigen_male, features_test, is_correct_function,
                                        num_eigenvectors=num_eigenvectors)

        features_test = get_features_filtered_by_label(X_test, y_test, 'M')
        is_correct_function = lambda x, y: x > y
        accuracy_male = test_function(eigen_female, eigen_male, features_test, is_correct_function,
                                     num_eigenvectors=num_eigenvectors)

#         print(f'f: {accuracy_female:.3f}, m: {accuracy_male:.3f}')
        accuracy_list.append((accuracy_female + accuracy_male) * .5)
        accuracy_list_female.append(accuracy_female)
        accuracy_list_male.append(accuracy_male)
        
    results['mean'] = np.mean(accuracy_list)
    results['mean_female'] = np.mean(accuracy_list_female)
    results['mean_male'] = np.mean(accuracy_list_male)
    
    results['std'] = np.std(accuracy_list)
    results['std_female'] = np.std(accuracy_list_female)
    results['std_male'] = np.std(accuracy_list_male)
    
    return results

def get_features_filtered_by_label(features, labels, selected_label):
    return [feature for feature, label in zip(features, labels) if label == selected_label]

def get_k_main_eigenvectors(X, k=1):
    m1 = X.mean(axis=0)
    corr_matrix = np.cov(X.T)
    corr_eigens = np.linalg.eigvalsh(corr_matrix)
    m2 = corr_matrix + np.outer(m1, m1) - corr_eigens[0] * np.eye(corr_matrix.shape[0])
    w, v = np.linalg.eigh(m2)
    return v[:, -k:].T

def get_name(path):
    return os.path.splitext(os.path.split(path)[1])[0]

# --- test functions ---

def test(eigen_female, eigen_male, features_test, is_correct_function, debug=False, **kwargs):
    counter = 0
    num_eigenvectors = kwargs['num_eigenvectors']
    for feature in features_test:
        eigen_test = get_k_main_eigenvectors(feature, k=num_eigenvectors)
        dist_to_female = min([np.square(ete - etr).sum() for etr, ete in zip(eigen_female, eigen_test)])
        dist_to_male = min([np.square(ete - etr).sum() for etr, ete in zip(eigen_male, eigen_test)])
        correct = is_correct_function(dist_to_female, dist_to_male)
        if correct:
            counter += 1
    if debug:
        print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    return counter / len(features_test)

def test_cosine(eigen_female, eigen_male, features_test, is_correct_function, debug=False, **kwargs):
    counter = 0
    for feature in features_test:
        dist_to_female = sum([min([np.dot(x, etr) / (np.linalg.norm(etr) * np.linalg.norm(x))
                                   for etr in eigen_female]) for x in feature])
        dist_to_male = sum([min([np.dot(x, etr) / (np.linalg.norm(etr) * np.linalg.norm(x))
                                   for etr in eigen_male]) for x in feature])
        correct = is_correct_function(dist_to_female, dist_to_male)
        if correct:
            counter += 1
    if debug:
        print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    return counter / len(features_test)


# -----------------------------------------------------------------
# def normalize(x):
#     return x - x.mean()

def normalize(x):
    return x / np.linalg.norm(x, ord=2, axis=1, keepdims=True)

# check normalize
def get_k_main_eigenvectors_svd(features, k=1):
    _, _, vh = np.linalg.svd(normalize(features), full_matrices=False)
    return vh[:k]

In [3]:
paths = sorted(glob.glob('datasets/khanty_4/*.wav'))

---

In [4]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=26).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 520/520 [02:09<00:00,  4.03it/s]


In [5]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=25)

In [6]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test, num_eigenvectors=1)
print(f"mean: {results['mean']:.3f} std: {results['std']:.3f}")

mean: 0.809 std: 0.035


In [7]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"mean: {results['mean']:.3f} std: {results['std']:.3f}")

mean: 0.830 std: 0.038


In [8]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"mean: {results['mean']:.3f} std: {results['std']:.3f}")

mean: 0.828 std: 0.038
